In [ ]:
import pandas as pd
import os
import ipywidgets as widgets # add import statement
from matplotlib import pyplot as plt
from scipy.signal import savgol_filter

In [ ]:
INTRO_TEXT = '''
<p><b>Curve Smoothing</b>
This tool is for smoothing and selecting land-ocean temperature data for visualization and download. Start by selecting a date
range, and then select the smoothing algorithm you want to use. Then click through to the next step, where you will change properies
of the curve smoothing algorithm you selected and visualize the data. In the final step, you may export the data you selected.
</p>
'''
SOURCES_TEXT = '''
<p>
<b>About Land-Ocean Temperature Data</b>
<a href="https://climate.nasa.gov/vital-signs/global-temperature/"
target="_blank">Global Temperature (NASA)</a>
,
<a href="https://data.giss.nasa.gov/gistemp/"
target="_blank">GISS Surface Temperature Analysis (NASA)</a>
</p><p>
This site is based on data downloaded from the following site on 2020-07-14:
<a href="https://data.giss.nasa.gov/gistemp/graphs/graph_data/Global_Mean_Estimates_based_on_Land_and_Ocean_Data/graph.txt"  # noqa
target="_blank">Global Mean Estimates based on Land and Ocean Data (NASA)</a>
'''
        
intro_text = widgets.HTML(value = INTRO_TEXT)
data_source_text = widgets.HTML(value = SOURCES_TEXT)

In [ ]:
intro_text

In [ ]:
data_source_text

In [ ]:
# Load data into memory from file
DATA_DIR = 'data'
DATA_FILE = 'land-ocean-temp-index.csv'

df = pd.read_csv(os.path.join(DATA_DIR, DATA_FILE), escapechar='#')

In [ ]:
slider = widgets.IntRangeSlider()
slider.max = max(df['Year']) # set the 'max' attribute of the slider to the minimum year of the our data  
slider.min = min(df['Year']) #   and let's do the same for 'min'

In [ ]:
output_df = widgets.Output()

In [ ]:
window_size = widgets.IntSlider(description = 'Window Size', min=1, max=100)
poly_order = widgets.IntSlider(description = 'Poly Order', min=0, max=10)

In [ ]:
output_plot = widgets.Output() # create an output widget called output_plot

In [ ]:
def on_slider_change(change):
    global selected # selected
    selected = df[(df['Year'] >= change['new'][0]) & (df['Year'] <= change['new'][1])]

In [ ]:
def display_df(change): 
    output_df.clear_output(wait=True)
    with output_df: 
        display(selected)

In [ ]:
def on_poly_order_change(change):
    global df, selected
    df['Savitzky-Golay'] = savgol_filter(df['Temperature'], window_size.value, change['new']) #df['Savitzky-Golay'] = savgol_filter(df['Temperature'], window_size, poly_order)
    selected = df[(df['Year'] >= slider.value[0]) & (df['Year'] <= slider.value[1])]

In [ ]:
def on_window_size_change(change):
    global df, selected, poly_order
    poly_order.max = min(change['new'] - 1, 10) # change the maximum of the poly_order widget
    df['Savitzky-Golay'] = savgol_filter(df['Temperature'], change['new'], poly_order.value)
    selected = df[(df['Year'] >= slider.value[0]) & (df['Year'] <= slider.value[1])]

In [ ]:
def display_plot(change): #def display_df(change): 
    output_plot.clear_output(wait=True) #output_df.clear_output(wait=True)
    with output_plot: 
        plt.xlabel('Year') # display(selected)
        plt.ylabel('Temperature') #
        plt.plot(selected['Year'], selected['Temperature']) #
        plt.plot(selected['Year'], selected['Savitzky-Golay']) # 
        plt.show() #

In [ ]:
slider.observe(on_slider_change, 'value')
slider.observe(display_df, 'value')

In [ ]:
poly_order.observe(on_poly_order_change, 'value')
poly_order.observe(display_df, 'value')

In [ ]:
window_size.observe(on_window_size_change, 'value')
window_size.observe(display_df, 'value')

In [ ]:
slider.value = (1890, 2000)
slider

In [ ]:
output_df

In [ ]:
window_size.value = 20
window_size

In [ ]:
poly_order.value = 3
poly_order

In [ ]:
slider.observe(display_plot, 'value')
window_size.observe(display_plot, 'value')
poly_order.observe(display_plot, 'value')

In [ ]:
display_plot(None)
output_plot